In [1]:
# This code was heavly based on https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/

import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from google.colab import drive
from PIL import Image
import os

In [28]:
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix/FID/mmb/'

filepath = '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix/FID/'
canti= '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix/FID/cantilever/'
mmb = '/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix/FID/mmb/'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks/Thesis_2022/TopOptPix2Pix/FID/mmb
ResNet50_MMB.png  ResUNet_MMB.png  UNet_MMB.png


In [3]:
# w11 = 437	h1 = 169
# w12 = 670	h2 = 403
# w21 = 733
# w22 = 966

def load_img(image_file):
  left_1 = 437
  left_2 = 733
  right_1 = 670
  right_2 = 966
  top = 169
  bottom = 403

  img = Image.open(image_file)
  img_1 = img.crop((left_1, top, right_1, bottom))
  # print("Shape 1: ", img_1.size)
  img_2 = img.crop((left_2, top, right_2, bottom))
  # print("Shape 2: ", img_1.size)
  return img_1, img_2

In [4]:
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

In [5]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [6]:
# define two fake collections of images
# images1 = randint(0, 255, 10*32*32*3)
# images1 = images1.reshape((10,32,32,3))
# images2 = randint(0, 255, 10*32*32*3)
# images2 = images2.reshape((10,32,32,3))

def fid(real, generated):
  # prepare the inception v3 model
  model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3)) # (299,299,3)

  print('Prepared', real.shape, generated.shape)
  # convert integer to floating point values
  real = real.astype('float32')
  generated = generated.astype('float32')
  # resize images
  real = scale_images(real, (256,256,3)) # (299,299,3)
  generated = scale_images(generated, (256,256,3))
  print('Scaled', real.shape, generated.shape)
  # pre-process images
  real = preprocess_input(real)
  generated = preprocess_input(generated)
  # fid between images1 and images1
  fid = calculate_fid(model, real, real)
  print('FID (same): %.3f' % fid)
  # fid between images1 and images2
  fid = calculate_fid(model, real, generated)
  print('FID (different): %.3f' % fid)
  return fid

In [29]:
scores = []
for results in os.listdir(mmb):
  # Getting the name of the method
  print(results)
  method = (os.path.splitext(results)[0])
  print('Method: ', method ) 
  # break
  # Getting the result image and splitting to real and generated
  real, gen = load_img(results)
  real = numpy.array(real)
  gen = numpy.array(gen)
  # Computing the FID Score
  score = fid(real, gen)
  scores.append((method, score)) 

ResNet50_MMB.png
Method:  ResNet50_MMB
Prepared (234, 233, 4) (234, 233, 4)
Scaled (234, 256, 256, 3) (234, 256, 256, 3)
FID (same): -0.000
FID (different): 9.167
ResUNet_MMB.png
Method:  ResUNet_MMB
Prepared (234, 233, 4) (234, 233, 4)
Scaled (234, 256, 256, 3) (234, 256, 256, 3)
FID (same): -0.000
FID (different): 13.176
UNet_MMB.png
Method:  UNet_MMB
Prepared (234, 233, 4) (234, 233, 4)
Scaled (234, 256, 256, 3) (234, 256, 256, 3)
FID (same): -0.000
FID (different): 28.925


In [30]:
print(*scores, sep = "\n")

('ResNet50_MMB', 9.167084949615834)
('ResUNet_MMB', 13.176245115541853)
('UNet_MMB', 28.924742522006134)


In [26]:
print(*canti_scores, sep = "\n")

('UNet_Canti', 26.428042979831034)
('ResNet50_Canti', 18.996844504551525)
('ResUNet_Canti', 14.153301748448685)
